# WorldCereal product download

This example illustrates the use of openEO for combining and downloading data from the [ESA WorldCereal](https://esa-worldcereal.org) project.

This project provides a global map of cereals for 2021 at 10m resolution! It can be used as an important base layer for agriculture use cases. Combined with the power of openEO, you can easily generate agricultural statistics over an area of interest, or use this data as a masking layer in an advanced workflow.

In this example, we'll illustrate a fairly simple case of combining two collections into a single image file.

In [2]:
import openeo

c = openeo.connect("openeo.vito.be").authenticate_oidc()

Authenticated using refresh token.


We list the available collection id's to make sure we have the right name.

In [3]:
[col["id"] for col in c.list_collections() if "WORLDCEREAL" in col["id"]]

['ESA_WORLDCEREAL_ACTIVECROPLAND',
 'ESA_WORLDCEREAL_IRRIGATION',
 'ESA_WORLDCEREAL_TEMPORARYCROPS',
 'ESA_WORLDCEREAL_WINTERCEREALS',
 'ESA_WORLDCEREAL_MAIZE',
 'ESA_WORLDCEREAL_SPRINGCEREALS']

In [4]:
c.describe_collection("ESA_WORLDCEREAL_MAIZE")

{'assets': {'metadata_iso_19139': {'href': 'https://services.terrascope.be/collectioncatalogue/srv/api/records/urn:eop:VITO:ESA_WORLDCEREAL_MAIZE_V1/formatters/xml?approved=true',
   'roles': ['metadata', 'iso-19139'],
   'title': 'ISO 19139 metadata',
   'type': 'application/vnd.iso.19139+xml'},
  'thumbnail': {'href': 'https://services.terrascope.be/collectioncatalogue/images/quicklooks/ESA_WORLDCEREAL_MAIZE_QL.png',
   'roles': ['thumbnail'],
   'title': 'Thumbnail',
   'type': 'image/jpeg'}},
 'cube:dimensions': {'bands': {'type': 'bands',
   'values': ['CLASSIFICATION', 'CONFIDENCE']},
  't': {'extent': ['2020-09-12T00:00:00Z', '2021-12-20T00:00:00Z'],
   'type': 'temporal'},
  'x': {'axis': 'x',
   'extent': [-180.00446428571428, 179.9955357],
   'reference_system': 4326,
   'step': 8.3333333333e-05,
   'type': 'spatial'},
  'y': {'axis': 'y',
   'extent': [-59.9955357, 80.0044643],
   'reference_system': 4326,
   'step': 8.3333333333e-05,
   'type': 'spatial'}},
 'description': 'WorldCereal maize, globally, at 10m resolution in specific seasons throughout the year. \n\n ##Seasons\n\n The date of observations in this collection is the end date of the season. This start date varies depending on the agro-ecological zone. WorldCereal defines these seasons:\n * wintercereals\n* maize main \n* maize second\n ',
 'extent': {'spatial': {'bbox': [[-180.00446428571428,
     -59.9955357,
     179.9955357,
     80.0044643]]},
  'temporal': {'interval': [['2020-09-12T00:00:00Z',
     '2021-12-20T00:00:00Z']]}},
 'id': 'ESA_WORLDCEREAL_MAIZE',
 'item_assets': {'classification': {'eo:bands': [{'description': 'classification band',
     'name': 'classification'},
    {'description': 'confidence band', 'name': 'confidence'}],
   'gsd': 10,
   'proj:epsg': 4326,
   'raster:bands': [{'classification:classes': [{'description': 'pixel not marked as maize',
       'name': 'not cropland',
       'value': 0},
      {'description': 'classified as maize in the respective season',
       'name': 'cropland',
       'value': 100}]},
    {}],
   'title': 'classification tiff',
   'type': 'image/tiff; application=geotiff; profile=cloud-optimized'}},
 'keywords': ['VITO',
  'COPERNICUS',
  'ESA',
  'WorldCereal',
  'Sentinel-1',
  'Sentinel-2',
  'Environment',
  'Maize',
  'Agriculture'],
 'license': 'CC-BY-4.0',
 'links': [{'href': 'https://esa-worldcereal.org',
   'rel': 'alternate',
   'title': 'Product web page.'},
  {'href': 'https://openeocloud.vito.be/openeo/1.0.0/collections',
   'rel': 'root'},
  {'href': 'https://openeocloud.vito.be/openeo/1.0.0/collections',
   'rel': 'parent'},
  {'href': 'https://openeocloud.vito.be/openeo/1.0.0/collections/ESA_WORLDCEREAL_MAIZE',
   'rel': 'self'}],
 'providers': [{'name': 'ESA',
   'roles': ['licensor'],
   'url': 'https://esa-worldcereal.org'},
  {'description': 'VITO remote sensing lead the consortium that produced the WorldCereal product on behalf of ESA. VITO also hosts the resulting archive.',
   'name': 'VITO',
   'processing:level': 'L4',
   'processing:software': {'WorldCereal': '1.0'},
   'roles': ['producer', 'host']}],
 'stac_extensions': ['https://stac-extensions.github.io/datacube/v2.2.0/schema.json',
  'https://stac-extensions.github.io/eo/v1.1.0/schema.json'],
 'stac_version': '0.9.0',
 'summaries': {'eo:bands': [{'bitPerValue': 8,
    'name': 'CLASSIFICATION',
    'offset': 0.0,
    'resolution': 10,
    'scaleFactor': 1.0,
    'title': 'CLASSIFICATION'},
   {'bitPerValue': 8,
    'name': 'CONFIDENCE',
    'offset': 0.0,
    'resolution': 10,
    'scaleFactor': 1.0,
    'title': 'CONFIDENCE'}],
  'federation:backends': ['vito'],
  'instruments': ['MSI', 'C-SAR'],
  'platform': ['Sentinel-1', 'Sentinel-2']},
 'title': 'ESA WorldCereal maize'}

In the following block, we combine two WorldCereal collections into a single output. Both the maize and the wintercereal collection will have value 100 for their respective crops. This means that once the two datacubes are merged, it's impossible to distinguish between the two. Therefore, a linear transformation is applied to the winter datacube. Furthermore, since both collections are mapped to a different point in time, the time dimension is reduced, such that the resulting map only contains one timestep with both crops shown.
The formula used here is just an example, and can be made much more complex depending on your use case.

In [5]:
extent = {'west': 3.0, 'south': 50.0, 'east': 4.0, 'north': 51.0, 'crs': 'EPSG:4326'}

temporal = ('2020-09-01T00:00:00Z', '2021-12-31T00:00:00Z')

maize = c.load_collection("ESA_WORLDCEREAL_MAIZE",
                         temporal_extent= temporal,
                         spatial_extent=extent,
                         bands=["CLASSIFICATION"]).reduce_dimension(dimension="t",reducer="mean")

winter = c.load_collection("ESA_WORLDCEREAL_WINTERCEREALS",
                         temporal_extent= temporal,
                         spatial_extent=extent,
                         bands=["CLASSIFICATION"]).apply(lambda x:100*(x+10)).reduce_dimension(dimension="t",reducer="mean")

combined = maize.merge_cubes(winter, overlap_resolver="sum")

We now have defined what openEO calls a 'process graph', but still need to execute it.
We will use an 'asynchronous' [batch job](https://open-eo.github.io/openeo-python-client/batch_jobs.html#batch-jobs) that gets sent to the server, as it can take a longer time to execute.

When finished, this command will automatically download the result from openEO to your local working directory.
You can also follow the progress and view results in the [openEO web editor](https://editor.openeo.cloud).

In [5]:
job = combined.execute_batch(
    title = "Worldcereal example Terrascope",
    out_format="GTiff",
)

0:00:00 Job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd': send 'start'
0:00:22 Job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd': queued (progress N/A)
0:00:28 Job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd': queued (progress N/A)
0:00:35 Job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd': queued (progress N/A)
0:00:43 Job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd': queued (progress N/A)
0:01:18 Job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd': queued (progress N/A)
0:01:31 Job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd': queued (progress N/A)
0:01:47 Job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd': queued (progress N/A)
0:02:06 Job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd': queued (progress N/A)
0:02:30 Job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd': queued (progress N/A)
0:03:00 Job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd': queued (progress N/A)
0:03:54 Job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd': running (progress N/A)
0:04:42 Job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd': running (progress N/A)
0:05:4

In [6]:
job.get_results().download_files()

<JobResults for job 'vito-j-0e6bd54acb9d4de2b6b6e03a5bb120dd'>